Привет, меня зовут Артем Хуршудов. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Отлично, что правильно использованы train/valid/test наборы.
* Выводы и рассуждения получились содержательными, их было интересно читать.
* Я оставил несколько советов, надеюсь, что они будут учтены в будущих проектах.
* Тебе удалось добиться очень хорошего качества, поздравляю!
* Проект может быть зачтен, но я его отправлю назад, чтобы у тебя была возможность задать вопросы и внести правки, при желании. Однако, ты можешь просто вернуть проект в таком же виде и я его зачту.

### <font color='orange'>Общее впечатление (ревью 2)</font>
* Проект зачтен!
* Удачи в дальнейшем обучении и следующих работах!

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Откроем и импортируем все известные библиотеки, пройденные на курсе. В общем - стандартный заголовок.

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
import timeit
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

Импортируем файл csv с использованием полный путей для облегчения проверяющего. Посмотрим, что там:

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')
data.sample(5)

,calls,minutes,messages,mb_used,is_ultra
74,63.0,427.64,93.0,19541.17,0
463,34.0,247.29,77.0,11925.40,0
1395,56.0,483.17,52.0,16331.71,0
1132,51.0,381.12,43.0,14771.91,0
1302,100.0,825.84,0.0,20273.82,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Изменим тип поля messages на int, потому что дробных сообщений быть не может:

In [4]:
data['messages']=data['messages'].astype("int")

Проверим на дубликаты.

In [5]:
data.duplicated().sum()

0

Проверим на корректность наименований столбцов (всё корректно):

In [6]:
data.columns

Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultra'], dtype='object')

Исследуем выборку: посмотрим на число пользователей тарифа "Ультра" и увидим, что она в пару раз меньше "Смарта"

In [7]:
data['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Проверим значения, содержащиеся в столбцах:

In [8]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


<b>Вывод:</b>
- Пропущенных значений нет
- Типы столбцов в порядке
- Дубликатов нет

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные загружены корректно, первичный осмотр проведен. Радует, что было отмечено наличие дисбаланса классов.
</div>

## Разбейте данные на выборки

Для начала разобьём данные на train и test с помощью изученной функции train_test_split (соответственно, 80 и 20 процентов):

In [9]:
train, test = train_test_split(data, test_size=0.4, random_state=12345)

In [10]:
features = test.drop('is_ultra', axis=1)
target = test['is_ultra']

Разобьём выборки на val(20%) и test(20%), при этом:
- features_train, target_train - обучающая выборка
- features_val, target_val - валидационная выборка
- features_test, target_test - тестовая выборка

In [11]:
features_val, features_test, target_val, target_test = train_test_split(features, target, test_size=0.5, random_state=12345)

In [12]:
features_train = train.drop('is_ultra', axis=1)
target_train = train['is_ultra']

<div class="alert alert-block alert-success">
<b>Успех:</b> Разбиение было сделано верно, молодец!
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Однако, желательно было напечатать размеры полученных наборов. Иногда, это помогает избежать ошибок.
</div>

In [13]:
features_train.shape

(1928, 4)

In [14]:
target_train.shape

(1928,)

<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Размеры выведены.
</div>

<div class="alert alert-block alert-success">
<b>Успех (ревью 2):</b> Спасибо!
</div>

## Исследуйте модели

### Дерево решений

Используем в цикле DecisionTreeClassifier для обучения дерева решений, выведем глубину и качество:

In [15]:
best_accuracy = 0
best_depth = 0
for depth in range(1,16):
    model = DecisionTreeClassifier(max_depth=depth, random_state=412)
    model.fit(features_train, target_train)
    accuracy = model.score(features_val, target_val)
    if accuracy > best_accuracy:
        # print("Глубина дерева:", depth,"    Качество:" ,accuracy)
        best_accuracy = accuracy
        best_depth = depth
print("Глубина дерева:", best_depth, "    Качество:", best_accuracy)

Глубина дерева: 3     Качество: 0.7853810264385692


<div class="alert alert-block alert-success">
<b>Успех:</b> Подбор параметров был реализован верно.
</div>

### Случайный лес

Теперь попробуем вариант "случайный лес", также выведем сравниваемые параметры:

In [16]:
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=412)
            model.fit(features_train, target_train)
            accuracy = model.score(features_val, target_val)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "    Количество деревьев:", best_est, "   мин. объектов в узле:", best_sample, "  Качество:", best_accuracy)

Глубина дерева: 7     Количество деревьев: 40    мин. объектов в узле: 3   Качество: 0.8040435458786936


<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что здесь подбиралось сразу несколько параметров.
</div>

### Логистическая регрессия

Подготовим модель логистической регрессии LogisticRegression:

In [17]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_val, target_val)
print("Качество:", accuracy)

Качество: 0.7589424572317263


<div class="alert alert-block alert-success">
<b>Успех:</b> Молодец, что попробовал несколько разных моделей!
</div>

## Проверьте модель на тестовой выборке

Проверим выбранную модель на тестовой выборке, последовательно по моделям:

In [18]:
model = DecisionTreeClassifier(max_depth=8, random_state=412)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7962674961119751


In [19]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7402799377916018


In [20]:
model = RandomForestClassifier(max_depth=7, n_estimators=40, min_samples_leaf=3, random_state=412)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7947122861586314


<b>Вывод:</b> Наилучшее качество (0.794) достигается при использовании randomforest с гиперпараметрами max_depth=7 , n_estimators=40 , min_samples_leaf=3

<div class="alert alert-block alert-success">
<b>Успех:</b> Тестирование было сделано абсолютно верно!
</div>

## (бонус) Проверьте модели на адекватность

Будем считать, что если модель предсказывает лучше, чем простое предположение, то количество перешедших на тариф "Смарт" и "Ультра", будет с такими же пропорциями, что и в исходной выборке - то она адекватна.

Посчитаем количество перешедших на тариф "Смарт" к общему количеству объектов в выборке.

In [21]:
(data['is_ultra']==0).sum() / data.shape[0]

0.693528313627878

Мы видим, что доля правильных ответов моделей выше, чем у нашего наивного предположения, считаем модели адекватными.

<div class="alert alert-block alert-success">
<b>Успех:</b> Именно это и есть проверка на адекватность.
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> В будущем советую реализовывать ее с помощью <a href="https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html"> DummyClassifier </a>. 
</div>

<b>Вывод: </b>Проверены три модели: деревья решений, логистическая регрессия, случайный лес.
Наилучшее качество (0.794) достигается при использовании randomforest с гиперпараметрами
max_depth=7 , n_estimators=40 , min_samples_leaf=3

<div class="alert alert-block alert-info">
<b>Совет: </b> Финальный вывод желательно было сделать подробнее.
</div>

<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Вариант c DummyClassifier ниже. Но качество прогноза при использовании RandomForest всё также выше, чем наивного прогноза, то есть результат длиннее и времени на него нужно тратить больше.
</div>

In [22]:
new_version = DummyClassifier(strategy="stratified", random_state=412)
new_version.fit(features_train, target_train)
my_accuracy = new_version.score(features_test, target_test)
print("Качество наивного прогноза:", my_accuracy)
print("Качество при использовании RandomForest:", accuracy)

Качество наивного прогноза: 0.5707620528771384
Качество при использовании RandomForest: 0.7947122861586314


In [23]:
new_version = DummyClassifier(strategy='most_frequent', random_state=412)
new_version.fit(features_train, target_train)
my_accuracy = new_version.score(features_test, target_test)
print("Качество наивного прогноза:", my_accuracy)
print("Качество при использовании RandomForest:", accuracy)

Качество наивного прогноза: 0.6842923794712286
Качество при использовании RandomForest: 0.7947122861586314


In [24]:
new_version = DummyClassifier(strategy='uniform', random_state=412)
new_version.fit(features_train, target_train)
my_accuracy = new_version.score(features_test, target_test)
print("Качество наивного прогноза:", my_accuracy)
print("Качество при использовании RandomForest:", accuracy)

Качество наивного прогноза: 0.48989113530326595
Качество при использовании RandomForest: 0.7947122861586314


<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Про окончательный вывод. "Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.". Соответственно, я могу лишь написать, что соответствующая модель построена, доля правильных ответов доведена до указанного значения, accuracy проверена. Наилучшее качество, как я уже писал, 0.794, достигается при использовании randomforest с гиперпараметрами max_depth=7 , n_estimators=40 , min_samples_leaf=3. Что ещё писать, честно говоря, не представляю.
</div>

<div class="alert alert-block alert-success">
<b>Успех (ревью 2):</b> Финальный вывод получился хорошо. Но я бы добавил еще информацию о том, сколько данных было, как ты их разделял на выборки итд.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
